In [6]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
import heapq
import numpy as np
from tqdm.auto import tqdm
import itertools

In [7]:

def getTrueElements(single_set):
    return np.sum(single_set)

def count_true_elements(frequencies):
    counts = {}
    
    for num_el, _  in frequencies:
        if num_el not in counts:
            counts[num_el] = 1
        else:
            counts[num_el] += 1
    return counts


PROBLEM_SIZE = 10
NUM_SETS = 20
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)

mydict = {i: [] for i in range(len(SETS[0]))}

for i, myset in enumerate(SETS):
    for j, val in enumerate(myset):
        if val:
            mydict[j].append(i)


State = namedtuple('State', ['taken', 'not_taken'])

In [8]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))

def distance(state):
    return len(state.taken)

def heuristic1(state): # heuristic that return the estimate to the goal
    newdict=mydict.copy()
    index_boolean=reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),)
    for i,val in enumerate(index_boolean):
        if(val):
            del newdict[i]

            
        
    
    
        
    n = len(newdict)
    combinations = []
    for i in range(1, n + 1):
        for subset in itertools.combinations(newdict, i):
            for j in range(2, i + 1):
                for subsubset in itertools.combinations(subset, j):
                    combinations.append(subsubset)
    combinations_sorted = sorted(combinations, key=len,reverse=True)
    h=0
    for combinazioni  in combinations_sorted:
        point_remaining=newdict.keys()
        
        
        point_totake=point_remaining-combinazioni
        
        
        expanded = [ newdict[point] for point in combinazioni]
        intersection = set(expanded[0]).intersection(*expanded[1:])
        if(intersection):
            
            h=len(point_remaining)-len(combinazioni) +1          
            break

    return h
        

        
          
        
         


        


In [9]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Probelm not solvable"

In [10]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((PROBLEM_SIZE, state))
counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            
            curr_tuple = (distance(new_state) + heuristic1(new_state), new_state)
            frontier.put(curr_tuple)
            pbar.update(1) 

        _, current_state = frontier.get()
        
    
print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)


58it [00:00, 1462.87it/s]

Solved in 3 steps (2 tiles)
